## Imports

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import time 
import statistics 

In [2]:
chart = pd.read_csv('../regional-global-daily-latest.csv', header = 1)
chart.head()

,Position,Track Name,Artist,Streams,URL
0,1,DÁKITI,Bad Bunny,6016657,https://open.spotify.com/track/4MzXwWMhyBbmu6h...
1,2,All I Want for Christmas Is You,Mariah Carey,5902919,https://open.spotify.com/track/0bYg9bo50gSsH3L...
2,3,Last Christmas,Wham!,5153859,https://open.spotify.com/track/2FRnf9qhLbvw8fu...
3,4,Santa Tell Me,Ariana Grande,4202883,https://open.spotify.com/track/0lizgQ7Qw35od7C...
4,5,It's Beginning to Look a Lot like Christmas,Michael Bublé,4074711,https://open.spotify.com/track/0lLdorYw7lVrJyd...


In [3]:
song_uris = []
for url in chart['URL']:
     song_uris.append(url[31:])
        
chart['song_uri'] = song_uris

In [4]:
chart.head(10)

,Position,Track Name,Artist,Streams,URL,song_uri
0,1,DÁKITI,Bad Bunny,6016657,https://open.spotify.com/track/4MzXwWMhyBbmu6h...,4MzXwWMhyBbmu6hOcLVD49
1,2,All I Want for Christmas Is You,Mariah Carey,5902919,https://open.spotify.com/track/0bYg9bo50gSsH3L...,0bYg9bo50gSsH3LtXe2SQn
2,3,Last Christmas,Wham!,5153859,https://open.spotify.com/track/2FRnf9qhLbvw8fu...,2FRnf9qhLbvw8fu4IBXx78
3,4,Santa Tell Me,Ariana Grande,4202883,https://open.spotify.com/track/0lizgQ7Qw35od7C...,0lizgQ7Qw35od7CYaoMBZb
4,5,It's Beginning to Look a Lot like Christmas,Michael Bublé,4074711,https://open.spotify.com/track/0lLdorYw7lVrJyd...,0lLdorYw7lVrJydTINhWdI
5,6,Rockin' Around The Christmas Tree,Brenda Lee,3866794,https://open.spotify.com/track/2EjXfH91m7f8HiJ...,2EjXfH91m7f8HiJN1yQg97
6,7,Jingle Bell Rock,Bobby Helms,3707604,https://open.spotify.com/track/7vQbuQcyTflfCIO...,7vQbuQcyTflfCIOu3Uzzya
7,8,positions,Ariana Grande,3578272,https://open.spotify.com/track/35mvY5S1H3J2QZy...,35mvY5S1H3J2QZyna3TFe0
8,9,Mood (feat. iann dior),24kGoldn,3518616,https://open.spotify.com/track/3tjFYV6RSFtuktY...,3tjFYV6RSFtuktYl3ZtYcq
9,10,It's the Most Wonderful Time of the Year,Andy Williams,3451295,https://open.spotify.com/track/1IcR6RlgvDczfvo...,1IcR6RlgvDczfvoWJSSY2A


## Credentials 

In [5]:
creds= pd.read_csv('../credentials')
SPOTIPY_CLIENT_ID = creds['Client ID'][0]
SPOTIPY_CLIENT_SECRET = creds['Client Secret'][0]
#Shameless redirect to my dance group if you're signing in for the first time
SPOTIPY_REDIRECT_URI= 'https://www.ballethermosoamanecer.com/'
username = ""

In [6]:
client_credentials_manager = SpotifyClientCredentials(client_id= SPOTIPY_CLIENT_ID, 
                                                      client_secret=SPOTIPY_CLIENT_SECRET)

sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)
scope = 'playlist-read-collaborative user-library-read playlist-read-private playlist-modify-public playlist-modify-private user-read-recently-played user-top-read'
token = util.prompt_for_user_token(username, scope, 
                                   client_id= SPOTIPY_CLIENT_ID, 
                                   client_secret= SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri= SPOTIPY_REDIRECT_URI)
if token:
    sp = spotipy.Spotify(auth=token)
    track_audio_features = []
    artist_uri = []
    artist_name = []
    artist_set = set()
    albums_set = set()
    art_uri = []
    loops = 0
    
    
    print(f'There are {len(song_uris)} songs that we found')
    while 20* (loops+1) <= len(song_uris):
        #get audio features in batches of 20
        track_audio_features.extend(sp.audio_features(song_uris[20*loops :20*(loops+1)]))
        
        #getting artist information 
        temp= [i for i in sp.tracks(song_uris[20*loops :20*(loops + 1)])['tracks'] ]
        for index in range(len(temp)):
            to_add = [artist['id'] for artist in temp[index]['artists']]
            artist_uri.append(to_add)
            artist_name.append([artist['name'] for artist in temp[index]['artists']])
            
            #if there was a collab, collect artist information
            if len(to_add) > 1:
                for artist in temp[index]['artists']:
                    artist_set.add(artist['id'])
        loops += 1
        print(f'Found audio features for {20*(loops+1)} songs')
        
    print('Creating Dataframe')
    audio_features = pd.DataFrame(track_audio_features)
    chart['artist_uri'] = artist_uri
    
    print("Merging Dataframe")
    df = chart.merge(audio_features, left_on = 'song_uri', right_on= 'id')
    

else:
    print("Can't get token for", username)
    
    
    

There are 200 songs that we found
Found audio features for 40 songs
Found audio features for 60 songs
Found audio features for 80 songs
Found audio features for 100 songs
Found audio features for 120 songs
Found audio features for 140 songs
Found audio features for 160 songs
Found audio features for 180 songs
Found audio features for 200 songs
Found audio features for 220 songs
Creating Dataframe
Merging Dataframe


In [ ]:
# type(sp.audio_features(sp.album(sp.artist_albums('4q3ewBCX7sLwd24euuV69X')['items'][0]['id'])['tracks']['items'][0]['id'])[0]
# )

In [7]:
client_credentials_manager = SpotifyClientCredentials(client_id= SPOTIPY_CLIENT_ID, 
                                                      client_secret=SPOTIPY_CLIENT_SECRET)

sp = spotipy.Spotify(client_credentials_manager= client_credentials_manager)
scope = 'playlist-read-collaborative user-library-read playlist-read-private playlist-modify-public playlist-modify-private user-read-recently-played user-top-read'
token = util.prompt_for_user_token(username, scope, 
                                   client_id= SPOTIPY_CLIENT_ID, 
                                   client_secret= SPOTIPY_CLIENT_SECRET, 
                                   redirect_uri= SPOTIPY_REDIRECT_URI)
#     ====================================================================
#                       Generating Artist's Features Data
#     ====================================================================

if token:
    a_count = 0 
    print(f'There are {len(artist_set)} artists that we found')
    aud_feat = []
    danceability = []
    energy = []
    key= []
    mode = []
    loudness = []
    speechiness = []
    acousticness = []
    instrumentalness = []   
    liveness = []
    valence = []
    tempo = []
    art_uri = []
    alb_uri = []
    alb_date = []
    alb_name = []
    alb_genre = []
    alb_group = []
    
    
    for uri in artist_set: 
        time.sleep(.1)
        for i in range(len(sp.artist_albums(uri)['items'])):
            art_uri.append(uri)
            alb_date.append(sp.artist_albums(uri)['items'][i]['release_date'])
            alb_name.append(sp.artist_albums(uri)['items'][i]['name']) 
            temp_uri = sp.artist_albums(uri)['items'][i]['id']
            temp_name = sp.artist_albums(uri)['items'][i]['name']
            alb_uri.append(temp_uri)
            albums_set.add(temp_uri)
            alb_group.append(sp.artist_albums(uri)['items'][i]['album_group'])
            if temp_uri in albums_set:
                print(f'Calculating album features for  {temp_name}' )
                l1 = []
                l2 = []
                l3 = []
                l4 = []
                l5 = []
                l6 = []
                l7 = []
                l8 = []
                l9 = []
                l10 = []
                l11 = []
                for j in range(len(sp.album((temp_uri))['tracks']['items'])):
                    feature = sp.audio_features(sp.album((temp_uri))['tracks']['items'][j]['id'])[0]
                    if type(feature) == dict:
                        l1.append(feature['danceability'])
                        l2.append(feature['energy'])
                        l3.append(feature['key'])
                        l4.append(feature['mode'])
                        l5.append(feature['loudness'])
                        l6.append(feature['speechiness'])
                        l7.append(feature['acousticness'])
                        l8.append(feature['instrumentalness'])
                        l9.append(feature['liveness'])
                        l10.append(feature['valence'])
                        l11.append(feature['tempo'])

                danceability.append(statistics.mean(l1))
                energy.append(statistics.mean(l2))
                loudness.append(statistics.mean(l5))
                speechiness.append(statistics.mean(l6))
                acousticness.append(statistics.mean(l7))
                instrumentalness.append(statistics.mean(l8))
                liveness.append(statistics.mean(l9))
                valence.append(statistics.mean(l10))
                tempo.append(statistics.mean(l11))
                key.append(statistics.median(l3))
                mode.append(statistics.median(l4))
            
            else: 
                print(f'Found a duplicate album: {temp_uri}. Will not calculate audio features')
                danceability.append('DUPLICATE')
                energy.append('DUPLICATE')
                key.append('DUPLICATE')
                mode.append('DUPLICATE')
                loudness.append('DUPLICATE')
                speechiness.append('DUPLICATE')
                acousticness.append('DUPLICATE')
                instrumentalness.append('DUPLICATE')   
                liveness.append('DUPLICATE')
                valence.append('DUPLICATE')
                tempo.append('DUPLICATE')
    
        a_count += 1 
        if a_count % 2 == 0:     
            print("==============================================")
            print(f'Found album features for {a_count} artists\n')
            print("==============================================")
            
    
    print("Creating Artist Dataframe")
    album_data = pd.DataFrame({'Artist URI': art_uri,
                               'Album URI': alb_uri,
                               'Album Group': alb_group,
                               'Album Name': alb_name,
                               'Album Release Date' : alb_date,
                               'Album Danceability' : danceability,
                               'Album Energy' : energy,
                               'Album Key' : key,
                               'Album Loudness' : loudness,
                               'Album Speechiness' : speechiness,
                               'Album Acousticness' : acousticness,
                               'Album Instrumentalness' : instrumentalness,
                               'Album Liveness' : liveness,
                               'Album Valence': valence,
                               'Album Tempo' : tempo
#                              'Album Generes': alb_genres
                          })
    

There are 129 artists that we found
Calculating album features for  Just Cause Y'all Waited 2 (Deluxe)
Calculating album features for  Just Cause Y'all Waited 2 (Deluxe)
Calculating album features for  Just Cause Y'all Waited 2
Calculating album features for  Just Cause Y'all Waited 2
Calculating album features for  Family Over Everything
Calculating album features for  Family Over Everything
Calculating album features for  Love Songs 4 The Streets 2
Calculating album features for  Love Songs 4 The Streets 2
Calculating album features for  Signed To The Streets 3
Calculating album features for  Signed To The Streets 3
Calculating album features for  Just Cause Y'all Waited
Calculating album features for  Just Cause Y'all Waited
Calculating album features for  Bloodas
Calculating album features for  Signed to the Streets 2.5
Calculating album features for  Love Songs for the Streets
Calculating album features for  Life Ain't No Joke
Calculating album features for  They Forgot
Calculatin

Calculating album features for  Ice Cream (with Selena Gomez)
Calculating album features for  Ice Cream (with Selena Gomez)
Calculating album features for  Ice Cream (with Selena Gomez)
Calculating album features for  Ice Cream (with Selena Gomez)
Calculating album features for  How You Like That
Calculating album features for  How You Like That
Calculating album features for  How You Like That
Calculating album features for  Sour Candy (with BLACKPINK)
Calculating album features for  DDU-DU DDU-DU (Remix JP Ver.)
Calculating album features for  KILL THIS LOVE (JP Ver.)
Found album features for 8 artists

Calculating album features for  Viva el Perreo
Calculating album features for  Lost In Time
Calculating album features for  La Alcaldia Del Perreo
Calculating album features for  Doxis Edition (The Mixtape)
Calculating album features for  Pre-Doxis (The Mixtape)
Calculating album features for  Sobredoxis
Calculating album features for  Sobredoxis
Calculating album features for  El Mom

Calculating album features for  X (Expanded Edition)
Calculating album features for  Fortune (Expanded Edition)
Calculating album features for  Fortune (Expanded Edition)
Calculating album features for  F.A.M.E. (Expanded Edition)
Calculating album features for  Head Shoulders Knees & Toes (feat. Norma Jean Martine) [Robin Schulz Remix]
Calculating album features for  Head Shoulders Knees & Toes (feat. Norma Jean Martine) [Robin Schulz Remix]
Calculating album features for  Head Shoulders Knees & Toes (feat. Norma Jean Martine) [Acoustic]
Calculating album features for  Head Shoulders Knees & Toes (feat. Norma Jean Martine) [Acoustic]
Calculating album features for  Cookie Dough
Calculating album features for  Head Shoulders Knees & Toes (feat. Norma Jean Martine) [Alle Farben Remix]
Calculating album features for  Head Shoulders Knees & Toes (feat. Norma Jean Martine) [Alle Farben Remix]
Calculating album features for  Head Shoulders Knees & Toes (feat. Norma Jean Martine)
Calculating

Calculating album features for  Pussy Talk (feat. Quavo, Lil Wayne, & Jack Harlow) [Remix]
Calculating album features for  Kitty Talk (feat. Quavo, Lil Wayne, & Jack Harlow) [Remix]
Calculating album features for  Too Blessed
Calculating album features for  Too Blessed
Calculating album features for  Lottery (Renegade) [Quavo Remix]
Calculating album features for  Lottery (Renegade) [Quavo Remix]
Calculating album features for  Open House (feat. Quavo)
Calculating album features for  Double Trouble (Quavo feat. Meek Mill)
Calculating album features for  A Million
Calculating album features for  A Million
Calculating album features for  Pose to Do (feat. French Montana & Quavo)
Calculating album features for  Broke Leg (with Quavo & Tyga)
Calculating album features for  Too Much Shaft (with Saweetie) [From Shaft: Original Motion Picture Soundtrack]
Calculating album features for  Bacc At It Again
Calculating album features for  B U B B L E G U M
Calculating album features for  L A M B T

Calculating album features for  God rest ye merry Gentlemen
Calculating album features for  Happy Holiday
Calculating album features for  Its Beginning To Look Like Christmas
Calculating album features for  Best of Christmas Songs
Calculating album features for  Mùa Giáng Sinh Năm Ấy
Calculating album features for  Christmas Chill 2020 - Calm & Cozy Xmas Songs
Calculating album features for  Best Christmas Songs
Calculating album features for  Acasă de Crăciun
Calculating album features for  Christmas Songs
Calculating album features for  Christmas Music
Calculating album features for  Holiday Songs
Calculating album features for  Best Christmas Songs 2020
Calculating album features for  100 Greatest Christmas Songs... Ever
Calculating album features for  Srećan Božić
Calculating album features for  Best Christmas Music
Calculating album features for  Festive Christmas Songs
Calculating album features for  Christmas Party Classics
Found album features for 30 artists

Calculating album 

...

Calculating album features for  Pink Friday: Roman Reloaded The Re-Up
Calculating album features for  Plastic Hearts
Calculating album features for  Plastic Hearts
Calculating album features for  Younger Now
Calculating album features for  Miley Cyrus & Her Dead Petz
Calculating album features for  Bangerz (Deluxe Version)
Calculating album features for  Bangerz (Deluxe Version)
Calculating album features for  Can't Be Tamed
Calculating album features for  The Time Of Our Lives
Calculating album features for  The Time Of Our Lives (International Version)
Calculating album features for  iTunes Live From London
Calculating album features for  Breakout
Calculating album features for  Breakout
Calculating album features for  Breakout
Calculating album features for  Breakout
Calculating album features for  Prisoner (feat. Dua Lipa)
Calculating album features for  Edge of Midnight (Midnight Sky Remix) (feat. Stevie Nicks)
Calculating album features for  Zombie (Live from the NIVA Save Our St

Calculating album features for  La Guerra de las Galaxias
Calculating album features for  John Williams in Vienna
Calculating album features for  Work From Home With John Williams
Calculating album features for  Star Wars: The Rise of Skywalker (Original Motion Picture Soundtrack)
Calculating album features for  Star Wars: The Rise of Skywalker (Original Motion Picture Soundtrack)
Calculating album features for  Star Wars: The Rise of Skywalker (Originalnyi saundtrek k filmu)
Calculating album features for  Star Wars: El ascenso de Skywalker (Banda Sonora Original)
Calculating album features for  Star Wars: Der Aufstieg Skywalkers (Original Film-Soundtrack)
Calculating album features for  Gwiezdne wojny: Skywalker Odrodzenie (Ścieżka Dźwiękowa z Filmu)
Calculating album features for  Star Wars & Return of the Jedi
Calculating album features for  Music From The Star Wars Saga - The Essential Collection
Calculating album features for  John Williams for Two Pianos - Indiana Jones
Calculat

Calculating album features for  Mitchell Ayres Plays Romantic Ballads For You (Digitally Remastered)
Calculating album features for  Have A Wonderful Weekend
Calculating album features for  Mitchell Ayres Plays Romantic Ballads for You
Calculating album features for  El cuento de las comadrejas (Original Motion Picture Soundtrack)
Calculating album features for  Canzoni di Natale
Calculating album features for  Kerstmuziek 2020 (Last Christmas)
Calculating album features for  Kerst 2020 (met Wham! en Mariah Carey)
Calculating album features for  Kerst Classics (It's Beginning To Look A Lot Like Christmas)
Calculating album features for  It's Beginning To Look A Lot Like Christmas
Calculating album features for  Christmas Music
Calculating album features for  Christmas Songs
Calculating album features for  Kerst Hits Top 100 (2020)
Calculating album features for  TOP 40 HITDOSSIER - Kerst Hits (Top 100)
Calculating album features for  Christmas Songs - Driving Home For Christmas
Calcula

Calculating album features for  Vroom Vroom
Calculating album features for  Vroom Vroom
Calculating album features for  Flood My Wrist (feat. Lil Uzi Vert)
Calculating album features for  Flood My Wrist (feat. Lil Uzi Vert)
Calculating album features for  Point Em Out
Calculating album features for  Point Em Out
Calculating album features for  King Of My City
Calculating album features for  Slide (Remix) (feat. Pop Smoke, A Boogie Wit da Hoodie & Chris Brown)
Found album features for 116 artists

Calculating album features for  Paradise (Cassian Remix)
Calculating album features for  Paradise (Loco Dice Remix)
Calculating album features for  Paradise
Calculating album features for  Wild (MEDUZA Remix)
Calculating album features for  Piece of Your Heart (DJ Phantasy Remix)
Calculating album features for  Discopolis 2.0 (MEDUZA Remix)
Calculating album features for  Discopolis 2.0 (MEDUZA Remix)
Calculating album features for  Discopolis 2.0 (MEDUZA Remix)
Calculating album features for 

Calculating album features for  What Christmas Means to Me (2014 CMA Country Christmas Performance)
Calculating album features for  Rare (Deluxe)
Calculating album features for  Rare (Deluxe)
Calculating album features for  Rare
Calculating album features for  Rare
Calculating album features for  Revival
Calculating album features for  Revival
Calculating album features for  Revival (Deluxe)
Calculating album features for  Revival (Deluxe)
Calculating album features for  Revival (Deluxe)
Calculating album features for  Stars Dance (Deluxe Edition)
Calculating album features for  Stars Dance
Calculating album features for  Stars Dance (Bonus Track Version)
Calculating album features for  Stars Dance
Calculating album features for  Selena x Votes
Calculating album features for  Ice Cream (with Selena Gomez)
Calculating album features for  Ice Cream (with Selena Gomez)
Calculating album features for  Ice Cream (with Selena Gomez)
Calculating album features for  Ice Cream (with Selena Gome

In [ ]:
# sp.audio_features(sp.album((temp_uri))['tracks']['items'][j]['id'])

In [ ]:
# sp.audio_features('2KdGrc6S5j96mQ7HjKzeHY')

In [ ]:
# album_data[album_data['Album URI'].duplicated()]

In [8]:
df

,Position,Track Name,Artist,Streams,URL,song_uri,artist_uri,danceability,energy,key,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,1,DÁKITI,Bad Bunny,6016657,https://open.spotify.com/track/4MzXwWMhyBbmu6h...,4MzXwWMhyBbmu6hOcLVD49,"[4q3ewBCX7sLwd24euuV69X, 0EFisYRi20PTADoJrifHrz]",0.731,0.573,4,...,0.1130,0.145,109.928,audio_features,4MzXwWMhyBbmu6hOcLVD49,spotify:track:4MzXwWMhyBbmu6hOcLVD49,https://api.spotify.com/v1/tracks/4MzXwWMhyBbm...,https://api.spotify.com/v1/audio-analysis/4MzX...,205090,4
1,2,All I Want for Christmas Is You,Mariah Carey,5902919,https://open.spotify.com/track/0bYg9bo50gSsH3L...,0bYg9bo50gSsH3LtXe2SQn,[4iHNK0tOyZPYnBU7nGAgpQ],0.336,0.627,7,...,0.0708,0.350,150.273,audio_features,0bYg9bo50gSsH3LtXe2SQn,spotify:track:0bYg9bo50gSsH3LtXe2SQn,https://api.spotify.com/v1/tracks/0bYg9bo50gSs...,https://api.spotify.com/v1/audio-analysis/0bYg...,241107,4
2,3,Last Christmas,Wham!,5153859,https://open.spotify.com/track/2FRnf9qhLbvw8fu...,2FRnf9qhLbvw8fu4IBXx78,[5lpH0xAS4fVfLkACg9DAuM],0.735,0.478,2,...,0.3550,0.947,107.682,audio_features,2FRnf9qhLbvw8fu4IBXx78,spotify:track:2FRnf9qhLbvw8fu4IBXx78,https://api.spotify.com/v1/tracks/2FRnf9qhLbvw...,https://api.spotify.com/v1/audio-analysis/2FRn...,262960,4
3,4,Santa Tell Me,Ariana Grande,4202883,https://open.spotify.com/track/0lizgQ7Qw35od7C...,0lizgQ7Qw35od7CYaoMBZb,[66CXWjxzNUsdJxJ2JdwvnR],0.525,0.621,7,...,0.2940,0.591,191.900,audio_features,0lizgQ7Qw35od7CYaoMBZb,spotify:track:0lizgQ7Qw35od7CYaoMBZb,https://api.spotify.com/v1/tracks/0lizgQ7Qw35o...,https://api.spotify.com/v1/audio-analysis/0liz...,204093,4
4,5,It's Beginning to Look a Lot like Christmas,Michael Bublé,4074711,https://open.spotify.com/track/0lLdorYw7lVrJyd...,0lLdorYw7lVrJydTINhWdI,[1GxkXlMwML1oSg5eLPiAz3],0.339,0.214,4,...,0.3410,0.363,94.775,audio_features,0lLdorYw7lVrJydTINhWdI,spotify:track:0lLdorYw7lVrJydTINhWdI,https://api.spotify.com/v1/tracks/0lLdorYw7lVr...,https://api.spotify.com/v1/audio-analysis/0lLd...,206640,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,One More Sleep,Leona Lewis,774806,https://open.spotify.com/track/78pn8k7RogKo2ox...,78pn8k7RogKo2oxl0DyX6d,[5lKZWd6HiSCLfnDGrq9RAm],0.638,0.813,11,...,0.3440,0.754,112.986,audio_features,78pn8k7RogKo2oxl0DyX6d,spotify:track:78pn8k7RogKo2oxl0DyX6d,https://api.spotify.com/v1/tracks/78pn8k7RogKo...,https://api.spotify.com/v1/audio-analysis/78pn...,239587,4
196,197,In Your Eyes,The Weeknd,771526,https://open.spotify.com/track/7szuecWAPwGoV1e...,7szuecWAPwGoV1e5vGu8tl,[1Xyo4u8uXC1ZmMpatF05PJ],0.667,0.719,7,...,0.0736,0.717,100.021,audio_features,7szuecWAPwGoV1e5vGu8tl,spotify:track:7szuecWAPwGoV1e5vGu8tl,https://api.spotify.com/v1/tracks/7szuecWAPwGo...,https://api.spotify.com/v1/audio-analysis/7szu...,237520,4
197,198,Bohemian Rhapsody - Remastered 2011,Queen,768966,https://open.spotify.com/track/4u7EnebtmKWzUH4...,4u7EnebtmKWzUH433cf5Qv,[1dfeR4HaWDbWqFHLkxsg1d],0.414,0.404,0,...,0.3000,0.224,71.105,audio_features,4u7EnebtmKWzUH433cf5Qv,spotify:track:4u7EnebtmKWzUH433cf5Qv,https://api.spotify.com/v1/tracks/4u7EnebtmKWz...,https://api.spotify.com/v1/audio-analysis/4u7E...,354320,4
198,199,Melek,Reynmen,766082,https://open.spotify.com/track/6XbuJHhqugUVdJb...,6XbuJHhqugUVdJbTdshUEL,[1Fg19vpUJpAAPFjnKaVYVW],0.569,0.642,11,...,0.2360,0.285,180.076,audio_features,6XbuJHhqugUVdJbTdshUEL,spotify:track:6XbuJHhqugUVdJbTdshUEL,https://api.spotify.com/v1/tracks/6XbuJHhqugUV...,https://api.spotify.com/v1/audio-analysis/6Xbu...,154923,4


In [ ]:
# df['collaborators'] = df['artist_uri'].apply(lambda x : len(x))

In [ ]:
# df[df['collaborators']>1]

In [ ]:
# df.drop(['URL', 'song_uri', 'type', 'uri', 'track_href'])

In [9]:
df.columns

Index(['Position', 'Track Name', 'Artist', 'Streams', 'URL', 'song_uri',
       'artist_uri', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature'],
      dtype='object')

In [10]:
album_data

,Artist URI,Album URI,Album Group,Album Name,Album Release Date,Album Danceability,Album Energy,Album Key,Album Loudness,Album Speechiness,Album Acousticness,Album Instrumentalness,Album Liveness,Album Valence,Album Tempo
0,3hcs9uc56yIGFCSy9leWe7,2lTzPbjHdZgNyV5cvhT7qE,album,Just Cause Y'all Waited 2 (Deluxe),2020-06-26,0.718136,0.561818,4.5,-8.220409,0.308618,0.096945,0.000022,0.141709,0.310832,124.316045
1,3hcs9uc56yIGFCSy9leWe7,0IRU1TnrUSfLrnGNUrEoAb,album,Just Cause Y'all Waited 2 (Deluxe),2020-06-25,0.716136,0.563591,4.0,-8.203636,0.292977,0.094682,0.000203,0.144168,0.297159,131.871864
2,3hcs9uc56yIGFCSy9leWe7,7ClxwGfXKhbDYykavKxvNY,album,Just Cause Y'all Waited 2,2020-05-08,0.724267,0.604933,2.0,-6.995133,0.282853,0.118413,0.000035,0.120633,0.311153,127.092000
3,3hcs9uc56yIGFCSy9leWe7,08CdWO2QxUfUcHDlwa8sRP,album,Just Cause Y'all Waited 2,2020-05-08,0.687533,0.605867,2.0,-6.991533,0.277807,0.113736,0.000329,0.120720,0.307713,126.000933
4,3hcs9uc56yIGFCSy9leWe7,74ruSB8Mj6Eu8TkN2Ryu9I,album,Family Over Everything,2019-12-11,0.758875,0.491312,7.5,-11.118313,0.282456,0.058115,0.000434,0.130656,0.386813,136.160625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2524,7iK8PXO48WeuP03g8YR51W,5XLW9FjUAB455F49Ec4weA,single,Me Gusta (with Cardi B & Myke Towers),2020-09-17,0.908000,0.604000,6.0,-4.286000,0.095000,0.428000,0.000000,0.091300,0.847000,110.124000
2525,7iK8PXO48WeuP03g8YR51W,5Un0k5MoNP9qALr7VcOVci,single,Jalapeño (Remix),2020-09-10,0.704000,0.641000,2.0,-5.794000,0.257000,0.280000,0.000000,0.120000,0.514000,84.862000
2526,7iK8PXO48WeuP03g8YR51W,3iH46vXV6xBilANZ2NBsFC,single,Enemigos Ocultos,2020-09-01,0.710000,0.845000,10.0,-3.296000,0.222000,0.220000,0.000000,0.416000,0.718000,172.087000
2527,7iK8PXO48WeuP03g8YR51W,3OFo9gRg3RbyRFFNc7JKQe,single,Luna,2020-08-28,0.787000,0.564000,7.0,-9.145000,0.057600,0.195000,0.000266,0.106000,0.258000,89.990000
